In [1]:
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import sys
import itertools
from collections import namedtuple, defaultdict
from pathlib import Path
from enum import Enum
from os.path import splitext

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

%load_ext autoreload
%autoreload 1
%aimport analyze

with open('plots/style.json') as f:
    mpl.rcParams.update(json.load(f))

# Results

## Wildstyle failures

In [3]:
ws = analyze.Analysis('results/wildstyle')
print(ws)

Analysis for Celeritas v0.4.4-rc.1 on wildstyle


In [4]:
ftab = analyze.make_failure_table(ws.failures())
ftab.to_frame()

,Failure
atlas-tilecal/orange+cpu (0),internal assertion failed: `volume` at `Orange...
atlas-tilecal/orange+cpu (1),internal assertion failed: `volume` at `Orange...
atlas-tilecal/orange+gpu (0),"`{""input"":{""_format"":""celer-sim"",""_version"":""0..."
atlas-tilecal/orange+gpu (1),"`{""input"":{""_format"":""celer-sim"",""_version"":""0..."
atlas-tilecal/vecgeom+cpu (0),(unknown failure)
atlas-tilecal/vecgeom+cpu (1),(unknown failure)
cms-hgcal/orange+cpu (0),(unknown failure)
cms-hgcal/orange+cpu (1),(unknown failure)
cms-hgcal/vecgeom+cpu (0),(unknown failure)
cms-hgcal/vecgeom+cpu (1),(unknown failure)


In [5]:
#ftab.to_dict()

In [6]:
ftab['cms2018+field+msc/vecgeom+cpu (0)']

'(unknown failure)'

Unconverged tracks (from runs that didn't fail):

In [7]:
summed = analyze.summarize_instances(ws.result[['unconverged']])
unconv = summed[('unconverged', 'mean')]
unconv[unconv > 0].unstack('arch') 

,arch
problem,geo


## Frontier

In [8]:
frontier = analyze.Analysis('results/frontier')
print(frontier)

Analysis for Celeritas v0.4.4-rc.2 on frontier


In [9]:
ftab = analyze.make_failure_table(frontier.failures())
ftab.to_frame()

,Failure
atlas-tilecal/orange+cpu (0),`srun: error: frontier10482: task 0: Segmentat...
atlas-tilecal/orange+cpu (1),`srun: error: frontier10482: task 0: Segmentat...
atlas-tilecal/orange+cpu (2),`srun: error: frontier10482: task 0: Segmentat...
atlas-tilecal/orange+cpu (3),`srun: error: frontier10482: task 0: Segmentat...
atlas-tilecal/orange+cpu (4),`srun: error: frontier10482: task 0: Segmentat...
atlas-tilecal/orange+cpu (5),`srun: error: frontier10482: task 0: Segmentat...
atlas-tilecal/orange+cpu (6),`srun: error: frontier10482: task 0: Segmentat...
atlas-tilecal/orange+cpu (7),`srun: error: frontier10482: task 0: Segmentat...
atlas-tilecal/orange+cpu+g4 (0),`srun: error: frontier10482: task 0: Exited wi...
atlas-tilecal/orange+cpu+g4 (1),`srun: error: frontier10482: task 0: Exited wi...


In [10]:
analyze.calc_geo_frac(frontier)

count      mean       std
problem                     geo    arch                           
cms-hgcal                   orange cpu     8.0  0.690889  0.002398
testem15                    orange cpu     8.0  0.429516  0.000810
testem15+field              orange cpu     8.0  0.542067  0.000487
                                   gpu     8.0  0.525380  0.000691
testem15+field+msc          orange cpu     8.0  0.640621  0.000480
testem3-composite+field+msc orange cpu     8.0  0.815128  0.001915
                                   gpu     8.0  0.885667  0.002270
testem3-composite+msc       orange cpu     8.0  0.791316  0.003756
testem3-flat                orange cpu     8.0  0.535109  0.000542
testem3-flat+field          orange cpu     8.0  0.616280  0.000855
testem3-flat+field+msc      orange cpu     8.0  0.764014  0.000508
                                   gpu     8.0  0.820114  0.000751
testem3-flat+msc            orange cpu     8.0  0.730158  0.000505

In [11]:
frontier.result["action_times"]

problem        geo     arch    instance
atlas-tilecal  geant4  g4      0           NaN
                               1           NaN
                               2           NaN
                               3           NaN
                               4           NaN
                                          ... 
testem3-flat   orange  gpu+g4  3           NaN
                               4           NaN
                               5           NaN
                               6           NaN
                               7           NaN
Name: action_times, Length: 480, dtype: object

## Perlmutter

In [12]:
perlmutter = analyze.Analysis('results/perlmutter')
print(perlmutter)

Analysis for Celeritas v0.4.3-7+0967ac32c on perlmutter


In [13]:
ftab = analyze.make_failure_table(perlmutter.failures())
ftab.to_frame()

,Failure
atlas-tilecal/orange+cpu (0),`srun: error: nid002801: task 0: Segmentation ...
atlas-tilecal/orange+cpu (1),`srun: error: nid002801: task 0: Segmentation ...
atlas-tilecal/orange+cpu (2),`srun: error: nid002801: task 0: Segmentation ...
atlas-tilecal/orange+cpu (3),`srun: error: nid002801: task 0: Segmentation ...
atlas-tilecal/orange+gpu (0),(unknown failure)
atlas-tilecal/orange+gpu (1),(unknown failure)
atlas-tilecal/orange+gpu (2),(unknown failure)
atlas-tilecal/orange+gpu (3),(unknown failure)


# Occupancy

In [15]:
problem = "testem3-flat"
kernel_stats = {
    "cuda/vecgeom": analyze.load_kernels(perlmutter, problem, "vecgeom"),
    "cuda/orange": analyze.load_kernels(perlmutter, problem, "orange"),
    "hip/orange": analyze.load_kernels(frontier, problem, "orange"),
}
ksdf = analyze.kernel_stats_dataframe(kernel_stats)

In [16]:
ksdf[ksdf['local_mem'] > 64]

const_mem  \
arch geo     name                                                     
cuda vecgeom along-step-general-linear-apply-eloss-fluct      20776   
     orange  initialize-tracks                                    0   
             along-step-general-linear-apply-eloss-fluct          0   
             photoel-livermore                                    0   
             annihil-2-gamma                                      0   
             geo-boundary                                         0   
hip  orange  initialize-tracks                                    0   
             geo-boundary                                         0   

                                                            heap_size  \
arch geo     name                                                       
cuda vecgeom along-step-general-linear-apply-eloss-fluct      8388608   
     orange  initialize-tracks                                8388608   
             along-step-general-linear-apply-eloss-fluct      8388608   
             photoel-livermore                                8388608   
             annihil-2-gamma                                  8388608   
             geo-boundary                                     8388608   
hip  orange  initialize-tracks                            68702699520   
             geo-boundary                                 68702699520   

                                                          local_mem  \
arch geo     name                                                     
cuda vecgeom along-step-general-linear-apply-eloss-fluct        104   
     orange  initialize-tracks                                  112   
             along-step-general-linear-apply-eloss-fluct        104   
             photoel-livermore                                   72   
             annihil-2-gamma                                     72   
             geo-boundary                                       152   
hip  orange  initialize-tracks                                  120   
             geo-boundary                                       120   

                                                          max_blocks_per_cu  \
arch geo     name                                                             
cuda vecgeom along-step-general-linear-apply-eloss-fluct                  3   
     orange  initialize-tracks                                            4   
             along-step-general-linear-apply-eloss-fluct                  3   
             photoel-livermore                                            3   
             annihil-2-gamma                                              4   
             geo-boundary                                                 4   
hip  orange  initialize-tracks                                            5   
             geo-boundary                                                 5   

                                                          max_threads_per_block  \
arch geo     name                                                                 
cuda vecgeom along-step-general-linear-apply-eloss-fluct                    256   
     orange  initialize-tracks                                              256   
             along-step-general-linear-apply-eloss-fluct                    256   
             photoel-livermore                                              256   
             annihil-2-gamma                                                256   
             geo-boundary                                                   256   
hip  orange  initialize-tracks                                              256   
             geo-boundary                                                   256   

                                                          max_warps_per_eu  \
arch geo     name                                                            
cuda vecgeom along-step-general-linear-apply-eloss-fluct                24   
     orange  initialize-tracks                   

# Geometric commplexity

In [53]:
with open('/Users/seth/Documents/work/celeritas-benchmarks/models/summary.json') as f:
    summaries = pd.DataFrame.from_records(json.load(f))
summaries.index = [splitext(fn)[0] for fn in summaries.pop('filename')]

In [62]:
abbreviate_problem = analyze.ProblemAbbreviator()
summaries['label'] = [abbreviate_problem({'geometry_name': k + ".gdml", 'enable_msc': True})
                      for k in summaries.index]

summaries['name'] = {
    "atlas-tilecal": "ATLAS TileCal",
    "cms-hgcal": "CMS HGCal",
    "cms-hllhc": "CMS Run 4 (HL)",
    "cms-run3": "CMS Run 3 (2022)",
    "cms2018": "CMS Run 2 (2018)",
    "dune10kt_v5_refactored_1x2x6": "DUNE A",
    "dunevd10kt_3view_30deg_v6_refactored_1x8x6": "DUNE B",
    "simple-cms": "Simple CMS",
    "testem15": "Infinite",
    "testem3-composite": "TestEm3 (composite)",
    "testem3-expanded": "TestEm3 (expanded)",
    "testem3-flat": "TestEM3 (flat)",
    "trackml": "TrackML",
    "zdcgeom": "ALICE ZDC",
    "atlas-emec": "ATLAS EMEC",
    "atlas-emec-nogaps": "ATLAS EMEC (filled gaps)",
}

In [63]:
summaries

,isotope,element,material,logical,physical,label,name
atlas-emec,25,9,12,281,40782,A$_\mathrm{E}$,ATLAS EMEC
atlas-emec-nogaps,25,9,12,353,80718,A$_\mathrm{E}^*$,ATLAS EMEC (filled gaps)
cms-hgcal,45,16,19,893,24506,C$_\mathrm{HG}$,CMS HGCal
atlas-tilecal,33,13,11,199,680,A$_\mathrm{T}$,ATLAS TileCal
cms-hllhc,111,37,452,10884,46009,C$_4$,CMS Run 4 (HL)
cms-run3,123,41,391,4147,27003,C$_3$,CMS Run 3 (2022)
cms2018,122,40,385,3815,19173,C$_2$,CMS Run 2 (2018)
dune10kt_v5_refactored_1x2x6,0,21,46,3831,6363,D$_A$,DUNE A
dunevd10kt_3view_30deg_v6_refactored_1x8x6,0,21,47,592,1145,D$_B$,DUNE B
simple-cms,20,6,6,7,7,T$_\mathrm{SC}$,Simple CMS


In [56]:
class Status(Enum):
    untested = [0,0,0,0.25]
    full = "tab:orange"
    nomsc = "tab:green"
    vecgeom = "tab:blue"

In [57]:
status = {
  "testem15": Status.full,
  "testem3-flat": Status.full,
  "testem3-composite": Status.full,
  "testem3-expanded": Status.full,
  "simple-cms": Status.full,
  "atlas-emec": Status.vecgeom,
  "atlas-tilecal": Status.vecgeom,
  "simple-cms": Status.full,
  "cms-hgcal": Status.nomsc,
  "cms2018": Status.vecgeom,
  "cms-run3": Status.untested,
  "cms-hllhc": Status.untested,
#  "dune10kt_v5_refactored_1x2x6": Status.untested,
#  "dunevd10kt_3view_30deg_v6_refactored_1x8x6": Status.untested,
  "trackml": Status.untested,
  "zdcgeom": Status.untested,
}

In [58]:
ps = summaries.loc[list(status.keys()),:]

In [64]:
(fig, ax) = plt.subplots(subplot_kw=dict(xscale='log', yscale='log'))
scat = ax.scatter([ps['logical'] + ps['physical']],
                  [ps['element'] + ps['material']],
                  marker='.', color = np.array([s.value for s in status.values()], dtype=object))
ax.set_xlabel("Logical + physical volumes")
ax.set_ylabel("Element + materials")
lo, hi = ax.get_xlim()
ax.set_xlim([lo, hi * 2])
ax.grid(which='major')
for lab, off, s in zip(ps['label'], scat.get_offsets(), status.values()):
    ax.annotate(lab, off, annotation_clip=False,
                color=([0,0,0,0.5] if status == Status.untested else 'k'))
fig.savefig("plots/problem-complexity.pdf", transparent=True)
plt.close()

In [43]:
temp = ps.loc[:, ['name', 'label']]
temp.index = temp.pop('name')
temp

,label
name,
Infinite,T$_\mathrm{inf}$
TestEM3 (flat),T$_3^-$
TestEm3 (composite),T$_3^+$
TestEm3 (expanded),T$_3^*$
Simple CMS,T$_\mathrm{SC}$
ATLAS EMEC,AE
ATLAS TileCal,AT
CMS HGCal,C$_\mathrm{HG}$
CMS Run 2 (2018),C$_2$


In [44]:
temp.to_clipboard()